<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project


In [2]:
!pip install transformers

     |████████████████████████████████| 665kB 2.8MB/s 
     |████████████████████████████████| 890kB 14.6MB/s 
     |████████████████████████████████| 1.1MB 15.9MB/s 
     |████████████████████████████████| 3.8MB 29.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e0eabdac6ca7a1715c2e89edb445df195576e28ea1c67bfde957464ee6daaa82
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!ls

Data		     emotion_model3_save  pos_neg_model2_save
emotion_model1_save  emotion_model4_save  pos_neg_model3_save
emotion_model2_save  pos_neg_model1_save  pos_neg_model4_save


In [4]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train.json
 data_sample_100.json
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.csv
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.csv
 fixed_valid.json
'informative words.ipynb'
 Raw
 test_fixed_processed_final.json
 test_fixed_processed_model1_final.csv
 test_fixed_processed_model2_final.csv
 test_fixed_processed_model3_final.csv
 test_fixed_processed_model4_final.csv
 valence_groupings.txt
 valid.csv
 valid_fixed_processed_final.json
 valid_fixed_processed_model1_final.csv
 valid_fixed_processed_model2_final.csv
 valid_fixed_processed_model3_final.csv
 valid_fixed_processed_model4_final.csv


In [0]:
dataset_name = "model2"
model_name = "model2"
model_type = "pos_neg"

In [39]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
import pandas as pd
import numpy as np
train_filename = "Data/data_sample_fixed_processed_" + dataset_name + "_final.csv"
valid_filename = "Data/valid_fixed_processed_" + dataset_name + "_final.csv"
test_filename = "Data/test_fixed_processed_" + dataset_name + "_final.csv"

train_df = pd.read_csv(train_filename, sep='|', error_bad_lines=False)
train_df = train_df.dropna()
valid_df = pd.read_csv(valid_filename, sep='|', error_bad_lines=False)
valid_df = valid_df.dropna()
test_df = pd.read_csv(test_filename, sep='|', error_bad_lines=False)
test_df = test_df.dropna()

In [0]:
category2index = {
    "invalid":0,
    "sentimental":1,
    "sad":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "proud":7,
    "afraid":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "guilty":13,
    "trusting":14,
    "nostalgic":15,
    "excited":16,
    "disgusted":17,
    "caring":18,
    "furious":19,
    "angry":20,
    "prepared":21,
    "disappointed":22,
    "faithful":23,
    "confident":24,
    "apprehensive":25,
    "jealous":26,
    "embarrassed":27,
    "devastated":28,
    "annoyed":29,
    "ashamed":30,
    "content":31,
    "anticipating":32
}
index2category = {}
for key, value in category2index.items():
  index2category[value] = key

def category2label(category):
  for key in category2index.keys():
    if key == category:
      return category2index[key]
  return 0

def label2emotion(labels):
  emotion_list = []
  for label in labels:
    emotion_list.append(index2category[label])
  return emotion_list

def label2category(labels):
  emotion_list = []
  for label in labels:
    if label == 0:
      emotion_list.append("negative")
    else:
      emotion_list.append("positive")
  return emotion_list

In [42]:
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
print('Number of valid sentences: {:,}\n'.format(valid_df.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

if model_type == "emotion":
  train_df['label'] = train_df['emotion'].apply(lambda x: int(category2label(x.lower())))
  valid_df['label'] = valid_df['emotion'].apply(lambda x: int(category2label(x.lower())))
  test_df['label'] = test_df['emotion'].apply(lambda x: int(category2label(x.lower())))
else:
  train_df['label'] = train_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)
  valid_df['label'] = valid_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)
  test_df['label'] = test_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 19,533

Number of valid sentences: 2,758

Number of test sentences: 2,538



,emotion,processed_emotion,text,vader_neg,vader_neu,vader_pos,vader_compound,textblob,flair_value,flair_confidence,label
15543,lonely,negative,All my friends have moved out of state after c...,0.031,0.812,0.157,0.7579,0.083333,POSITIVE,0.964614,0
6988,angry,negative,I'm tired of being harassed by this corrupt ci...,0.307,0.641,0.051,-0.9429,-0.240000,NEGATIVE,0.973983,0
18041,afraid,negative,I had to investigate a noise coming from my b...,0.091,0.909,0.000,-0.2500,0.000000,POSITIVE,0.675280,0
3392,apprehensive,negative,My husband says he stopped drinking but I'm re...,0.306,0.694,0.000,-0.7783,0.000000,NEGATIVE,0.806775,0
1822,hopeful,positive,My wife is trying to get a job at the hospital...,0.000,1.000,0.000,0.0000,0.000000,POSITIVE,0.978469,1
12018,furious,negative,so mad! my cat destroyed my favourite jacket S...,0.413,0.587,0.000,-0.9639,-0.475379,POSITIVE,0.999671,0
14479,anxious,negative,I am feeling fear these days. I am feeling an ...,0.362,0.395,0.243,-0.6249,0.200000,NEGATIVE,0.779193,0
9691,devastated,negative,My cat is sick_comma_ I don't know what to do....,0.082,0.845,0.073,-0.1010,-0.928571,NEGATIVE,0.998624,0
4538,proud,positive,When my friend made the basketball team. He wo...,0.068,0.692,0.240,0.6934,0.254167,POSITIVE,0.995331,1
7714,furious,negative,My mother in law does not know how to mind her...,0.066,0.934,0.000,-0.5009,0.000000,NEGATIVE,0.722510,0


In [0]:
sentences_train = train_df.text.values
labels_train = train_df.label.values
sentences_valid = valid_df.text.values
labels_valid = valid_df.label.values
sentences_test = test_df.text.values
labels_test = test_df.label.values

In [44]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

longer_list = []
for index, sent in enumerate(sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
sentences_train = np.delete(sentences_train, longer_list)
labels_train = np.delete(labels_train, longer_list)

def tokenize(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.

      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 256,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.from_numpy(labels)

  return(input_ids, attention_masks, labels)

input_ids_train, attention_masks_train, labels_train = tokenize(sentences_train, labels_train)
input_ids_valid, attention_masks_valid, labels_valid = tokenize(sentences_valid, labels_valid)
input_ids_test, attention_masks_test, labels_test = tokenize(sentences_test, labels_test)

Loading BERT tokenizer...


In [45]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_valid = TensorDataset(input_ids_valid, attention_masks_valid, labels_valid)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

#Use the given training and validation datasets
train_dataset, val_dataset = dataset_train, dataset_valid

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))
print('{:>5,} test samples'.format(len(dataset_test)))

19,533 training samples
2,758 validation samples
2,538 test samples


In [0]:
from transformers import BertForSequenceClassification, BertTokenizer
output_dir = "./" + model_type + "_" + model_name + "_save/"

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

print(model_type + "_" + model_name + " loaded...")

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

def predict(dataset):

  # Create the DataLoader for  dataset.
  prediction_data = dataset
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


  # Prediction on test set

  print('Predicting labels for {:,} sentences...'.format(len(dataset)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print('DONE.')
  return predictions, true_labels

test_predictions, test_true_labels = predict(dataset_test)
valid_predictions, valid_true_labels = predict(dataset_valid)
train_predictions, train_true_labels = predict(dataset_train)

Predicting labels for 2,542 sentences...
DONE.
Predicting labels for 2,763 sentences...
DONE.
Predicting labels for 19,533 sentences...
DONE.


In [32]:
from sklearn.metrics import classification_report, accuracy_score

def metrics(predictions, true_labels):
  for i in range(len(predictions)):
    if i == 0:
      pred_flat = np.argmax(predictions[i], axis=1).flatten()
      confidence_flat = np.amax(predictions[i], axis=1).flatten()
      labels_flat = true_labels[i].flatten()
    else:
      pred_flat = np.append(pred_flat, np.argmax(predictions[i], axis=1).flatten())
      confidence_flat = np.append(confidence_flat, np.amax(predictions[i], axis=1).flatten())
      labels_flat = np.append(labels_flat, true_labels[i].flatten())

  if model_type == "emotion":
    pred_flat = label2emotion(pred_flat)
    labels_flat = label2emotion(labels_flat)
  else:
    pred_flat = label2category(pred_flat)
    labels_flat = label2category(labels_flat)
  print("Classwise Report")
  print(classification_report(labels_flat, pred_flat))
  return pred_flat, confidence_flat, labels_flat
print("Metrics on testing set")
pred_flat_test, confidence_flat_test, labels_flat_test = metrics(test_predictions, test_true_labels)
print("Metrics on validation set")
pred_flat_valid, confidence_flat_valid, labels_flat_valid = metrics(valid_predictions, valid_true_labels)
print("Metrics on training set")
pred_flat_train, confidence_flat_train, labels_flat_train = metrics(train_predictions, train_true_labels)

Metrics on testing set
Classwise Report
              precision    recall  f1-score   support

      afraid       0.33      0.40      0.36        73
       angry       0.32      0.29      0.30        84
     annoyed       0.61      0.64      0.62        91
anticipating       0.39      0.40      0.39        73
     anxious       0.39      0.44      0.41        78
apprehensive       0.43      0.41      0.42        71
     ashamed       0.33      0.25      0.28        64
      caring       0.72      0.60      0.65        80
   confident       0.57      0.63      0.59        75
     content       0.63      0.68      0.65        76
  devastated       0.55      0.52      0.53        67
disappointed       0.51      0.53      0.52        81
   disgusted       0.69      0.80      0.74        86
 embarrassed       0.76      0.69      0.72        81
     excited       0.45      0.43      0.44        91
    faithful       0.67      0.66      0.67        50
     furious       0.45      0.37      0.

In [33]:
#final_train_df = pd.DataFrame(sentences_train, columns=['sentences'])
final_train_df['emotion_actual'] = pd.DataFrame(labels_flat_train, columns=['emotion_actual'])
final_train_df['emotion_pred'] = pd.DataFrame(pred_flat_train, columns=['emotion_pred'])
final_train_df

,sentences,pos_neg_actual,pos_neg_pred,emotion_actual,emotion_pred
0,I remember going to the fireworks with my best...,positive,positive,sentimental,sentimental
1,One year during christmas_comma_ i did not get...,negative,negative,sad,sad
2,I have been in college for four years and am o...,positive,positive,hopeful,hopeful
3,You never know how happy you can be until you try,positive,positive,joyful,joyful
4,the redsox have a chance to break the amount o...,positive,positive,impressed,impressed
...,...,...,...,...,...
19528,living out in the country when it gets dark it...,negative,negative,afraid,afraid
19529,I took my boots off after a long day of workin...,positive,positive,trusting,trusting
19530,Anytime I see a mall_comma_ I get nostalgic ab...,positive,positive,nostalgic,nostalgic
19531,i cant wait to play the new madden_comma_ ive ...,positive,positive,anticipating,anticipating


In [0]:
#final_valid_df = pd.DataFrame(sentences_valid, columns=['sentences'])
final_valid_df['emotion_actual'] = pd.DataFrame(labels_flat_valid, columns=['emotion_actual'])
final_valid_df['emotion_pred'] = pd.DataFrame(pred_flat_valid, columns=['emotion_pred'])
#final_test_df = pd.DataFrame(sentences_test, columns=['sentences'])
final_test_df['emotion_actual'] = pd.DataFrame(labels_flat_test, columns=['emotion_actual'])
final_test_df['emotion_pred'] = pd.DataFrame(pred_flat_test, columns=['emotion_pred'])

In [0]:
final_train_df.to_csv("Data/" + model_type + "_" + model_name + "_" + "predictions_train.csv", index=False)
final_valid_df.to_csv("Data/" + model_type + "_" + model_name + "_" + "predictions_valid.csv", index=False)
final_test_df.to_csv("Data/" + model_type + "_" + model_name + "_" + "predictions_test.csv", index=False)